# 关于正则表达式


https://docs.python.org/3/howto/regex.html#regex-howto

In [1]:
import re

## matching characters

```python
. ^ $ * + ? { } [ ] \ | ( )
```

`[ ] ` specify a character class.

元字符在方括号内成为本身的含义，不具有其他特别的功能。

`[^ ]` 表示匹配除去对应字符之外的内容。但是当 `^` 不是紧挨着括号时，就不具有这一功能，而变为匹配这个字符的含义。

转义字符的标志 `\`

`*` 匹配方式的贪婪模式。

In [2]:
pattern = r'a[bcd]*b'
strs = 'abcbd'
re.match(pattern, strs)

<re.Match object; span=(0, 4), match='abcb'>

In [3]:
pattern = r'a[bcd]*'
strs = 'abcbd'
re.match(pattern, strs)

<re.Match object; span=(0, 5), match='abcbd'>

In [10]:
pattern = r'a[bcd]*b'
strs = 'abced'
re.match(pattern, strs)

<re.Match object; span=(0, 2), match='ab'>

`{m,n}`

In [11]:
pattern = r'a/{1,3}b'
strs = 'a///b'
re.match(pattern, strs)

<re.Match object; span=(0, 5), match='a///b'>

In [12]:
p = re.compile('ab*')
p

re.compile(r'ab*', re.UNICODE)

In [13]:
p = re.compile('ab*', re.IGNORECASE)
p

re.compile(r'ab*', re.IGNORECASE|re.UNICODE)

## the backslash plague

python 以字符串的形式处理正则表达式的优势的同时，也导致了一些问题。

就是转义字符的 `\` 的处理，应对方法是使用 raw string。

In [14]:
p = re.compile(r'[a-z]+')
p

re.compile(r'[a-z]+', re.UNICODE)

In [20]:
p.match('')

In [21]:
print(p.match(''))

None


In [24]:
m = p.match('tempo')

In [25]:
m.group()

'tempo'

In [26]:
m.start()

0

In [27]:
m.end()

5

In [28]:
m.span()

(0, 5)

In [40]:
m = p.search('::: message'); print(m)

<re.Match object; span=(4, 11), match='message'>


In [41]:
m.group()

'message'

In [42]:
m.span()

(4, 11)

```python
p = re.compile(...)
m = p.match('string goes here')
if m:
    print('Match found: ', m.group())
else:
    print('No match')
```

返回所有匹配结果。

In [43]:
p = re.compile(r'\d+')
p.findall('12 drummers drumming, 11 pippers piping, 10 lords 10 lords a leaping')

['12', '11', '10', '10']

In [47]:
iterator = p.finditer('12 drummers drumming, 11 ... 10 ...')
iterator

In [45]:
for match in iterator:
    print(match.span())

(0, 2)
(22, 24)
(29, 31)


In [48]:
for match in iterator:
    print(match.group())

12
11
10


In [50]:
print(re.match(r'From\s+', 'Fromage amk'))

None


In [51]:
re.match(r'From\s+', 'From amk Thu May 14 19:12:10 1998')

<re.Match object; span=(0, 5), match='From '>

关于使用模块级方法（`re.method()`）或者自己编译正则表达模式（`pattern =  ...`）的建议：

> 如果是在循环中调用，预先编译会节省一些函数调用。在循环之外，两者没多大区别。

## compliation flags

flags 们可以用全名和对应的一个字母的缩写表示。

如果想用多个 flag，那不同 flag 之间用 `|` 表示，这是**按位与**的符号。

以下是一些可用的 flags：

`ASCII, A`    让部分转义字符仅匹配 ASCII 码，默认会匹配unicode码。

`DOTALL, S`   让 `.` 匹配包括换行符在内的所有字符，默认是不匹配换行符的。

`IGNORECASE, I`    忽略大小写，默认会区分大小写进行匹配。

`LOCALE, L`    地点匹配，即与一国所用语言、表述方式有关。

`MULTILINE, M`    多行匹配，此时`^` 会匹配字符串开头和每行开头，`$` 会匹配字符串末尾和每行行尾。

`VERBOSE, X`    冗长匹配，更清晰易懂。【我不懂了】，有助于书写更易读的正则表达式。这种模式下，空格被忽略（当然，除去字符类或者转义字符之后的空格）。你还可以在表达式中写注释。例如：

In [4]:
import re

charref = re.compile(r"""
&[#]                 # start of a numeric entity reference
(
    0[0-7]+          # octal form
  | [0-9]+           # decimal form
  | x[0-9a-fA_F]+    # hexadecimal form
)
;                    # trailing semicolon 结尾符号
""", re.VERBOSE)

如果没有 verbose 标志，这个表达式应该长这个样子：

In [9]:
charref = re.compile("&#([0-7]+"
                     "|[0-9]+"
                     "|x[0-9a-fA-F]+);")

In [10]:
charref.match('&#74586;')

<re.Match object; span=(0, 8), match='&#74586;'>

## more pattern power

zero-width assertions. 不会重复的判断，比如，字符边界 `\b`。这一判断不会改变位置。

`^ $` 在明确多行flag的情况下会自动匹配每行，在多行flag没有给定之后，才会只判断字符串开头和结尾。

In [15]:
strs = """
When i was just a little girl
i ask my mother
what will i be
will i be pretty
will i be reach"""

pattern = re.compile(r'^w', re.MULTILINE)
pattern.findall(strs)

['w', 'w', 'w']

In [16]:
strs = """
When i was just a little girl
i ask my mother
what will i be
will i be pretty
will i be reach"""

pattern = re.compile(r'^w')
pattern.findall(strs)

[]

In [17]:
strs = """
When i was just a little girl
i ask my mother
what will i be
will i be pretty
will i be reach"""

pattern = re.compile(r'\w$', re.MULTILINE)
pattern.findall(strs)

['l', 'r', 'e', 'y', 'h']

In [18]:
strs = """
When i was just a little girl
i ask my mother
what will i be
will i be pretty
will i be reach"""

pattern = re.compile(r'\w$')
pattern.findall(strs)

['h']

In [22]:
strs = """
When i was just a little girl
i ask my mother
what will i be
will i be pretty
will i be reach"""

pattern = re.compile(r'\A')
pattern.findall(strs)

['']

In [20]:
strs = """
When i was just a little girl
i ask my mother
what will i be
will i be pretty
will i be reach"""

pattern = re.compile(r'\Aw')
pattern.findall(strs)

[]

In [21]:
strs = """
When i was just a little girl
i ask my mother
what will i be
will i be pretty
will i be reach"""

pattern = re.compile(r'\Aw', re.MULTILINE)
pattern.findall(strs)

[]

In [23]:
strs = """
When i was just a little girl
i ask my mother
what will i be
will i be pretty
will i be reach"""

pattern = re.compile(r'\A', re.MULTILINE)
pattern.findall(strs)

['']

`\b` 单词边界，单词边界包括空格和非字母字符。

也是一个零宽字符。

In [24]:
p = re.compile(r'\bclass\b')

p.search('no class at all')

<re.Match object; span=(3, 8), match='class'>

In [25]:
p.search('the declassified algorithm')

In [26]:
p.search('the-class-algorithm')

<re.Match object; span=(4, 9), match='class'>

In [28]:
p = re.compile(r'\bclass')
p.search('the-classalgorithm')

<re.Match object; span=(4, 9), match='class'>

需要注意的是，`\b` 在python中是退格键的符号，ascii码为8。所以在有这个符号的时候如果不用 raw string，就会出问题。下面的例子

In [30]:
print('\bclass\b')

class


In [31]:
p = re.compile('\bclass\b')
p.search('no class at all')

## grouping

分组用圆括号。`( )`

In [1]:
import re

p = re.compile('(ab)*')
p.match('ababababababa').span()

(0, 12)

In [5]:
p = re.compile('(a)b')
m = p.match('ab')
m.group()

'ab'

In [6]:
m.group(0)

'ab'

In [9]:
p = re.compile('(a)+b')
m = p.match('aaaaabababab')
m.group()

'aaaaab'

In [10]:
m.group(0)

'aaaaab'

In [14]:
m.group(1)

'a'

In [16]:
m.group(2)

IndexError: no such group

正则表达式中 group 是可以嵌套的。`group(数字)` 是匹配得到的字符，0 表示匹配到的整个字符串，1 表示从左往右第一个括号，依次类推。

In [17]:
p = re.compile('(a(b)c)d')
m = p.match('abcd')
m.group()

'abcd'

In [18]:
m.group(0)

'abcd'

In [19]:
m.group(1)

'abc'

In [20]:
m.group(2)

'b'

In [21]:
m,group(3)

NameError: name 'group' is not defined

In [22]:
m.group(0, 1, 2, 1, 2)

('abcd', 'abc', 'b', 'abc', 'b')

In [23]:
m.groups()

('abc', 'b')

`m.groups()` 返回的是从编号为 1 到最大编号的匹配组。

匹配到的表达式如何引用？用 `\1 \2` 这种形式。

In [24]:
p = re.compile(r'\b(\w+)\s+\1\b')
p.search('Paris in the the spring').group()
# 匹配一个重复一次的单词

'the the'

In [25]:
p.search('Paris in the the spring')

<re.Match object; span=(9, 16), match='the the'>

In [26]:
p.search('Paris in the      the the spring')

<re.Match object; span=(9, 21), match='the      the'>

In [27]:
p.search('Paris in thethe spring')

In [28]:
p.search('Paris in the the spring').groups()

('the',)

## non-capturing and named groups

`(?P )` 问号后以 P 开头的，是python在perl基础上增加的扩展。

非捕获分组 `(?:...)`

In [35]:
m = re.match("([abc])+", 'abc')
m.groups()

('c',)

In [36]:
m

<re.Match object; span=(0, 3), match='abc'>

> 这里为什么 group 里面是 c 而不是 abc？

In [39]:
m.group(0)

'abc'

In [38]:
m.group(1)

'c'

In [30]:
m = re.match("(?:[abc])+", "abc")
m.groups()

()

可以给名字的分组

`(?P<name>...)`

In [40]:
p = re.compile(r'(?P<word>\b\w+\b)')
m = p.search('((((Losts of punctuation)))')
m.group('word')

'Losts'

In [41]:
m.group(0)

'Losts'

In [42]:
m.group(1)

'Losts'

`groupdict()`

In [43]:
m = re.match(r'(?P<fisrt>\w+) (?P<last>\w)', 'Jane Doe')
m.groupdict()

{'fisrt': 'Jane', 'last': 'D'}

In [44]:
m.groups()

('Jane', 'D')

In [45]:
m = re.match(r'(?P<fisrt>\w+) (?P<last>\w+)', 'Jane Doe')
m.groupdict()

{'fisrt': 'Jane', 'last': 'Doe'}

In [46]:
m = re.match(r'(?P<fisrt>\w+) (?P<last>\w)', 'Jane Doe')
m.groups()

('Jane', 'D')

In [47]:
m.group(0)

'Jane D'

In [48]:
m.group(1)

'Jane'

In [49]:
m.group(2)

'D'

In [50]:
m.group(3)

IndexError: no such group

In [51]:
m = re.match(r'(?P<fisrt>\w+) (?P<last>\w+)', 'Jane Doe')
m.groups()

('Jane', 'Doe')

In [52]:
m.group(0)

'Jane Doe'

In [53]:
m.group(1)

'Jane'

In [54]:
m.group(2)

'Doe'

In [55]:
m.group(3)

IndexError: no such group

back reference 即引用前面匹配到的结果。

可以用 `\1` 这样的数字，也可以用 `(P=nme)` 这种python独有的形式。

In [56]:
p = re.compile(r'\b(?P<word>\w+)\s+(?P=word)\b')
p.search('Paris in the the spring').group()

'the the'

In [57]:
p = re.compile(r'\b(\w+)\s+\1\b')
p.search('paris in the the sprint').group()

'the the'

## lookahead assertions

向前看的断言语句，即判断这个位置之前还判断一下它前面的内容。所谓前面，是指没有判断过的地方。【这是目前为止自己的理解】

positive form `(?=...)`


negative form `(?!...)`

In [58]:
files = """
foo.bar
autoexec.bat
sendmail.cf
printers.conf
"""

In [63]:
p = re.compile(r".*[.].*$", re.MULTILINE)
p.findall(files)

['foo.bar', 'autoexec.bat', 'sendmail.cf', 'printers.conf']

In [61]:
p = re.compile(r".*[.][^b].*$", re.MULTILINE)
p.findall(files)

['sendmail.cf', 'printers.conf']

In [64]:
p = re.compile(r".*[.][^(bat)].*$", re.MULTILINE)
p.findall(files)

['sendmail.cf', 'printers.conf']

In [65]:
p = re.compile(r".*[.]([^b]..|.[^a].|..[^t])$", re.MULTILINE)
p.findall(files)

['bar']

In [66]:
p = re.compile(r".*[.]([^b]..|.[^a].|..[^t])$", re.MULTILINE)
p.findall(files)

['bar']

In [74]:
re.match(r".*[.]([^b]..|.[^a].|..[^t])$", 'foo.bar')

<re.Match object; span=(0, 7), match='foo.bar'>

In [75]:
re.match(r".*[.]([^b]..|.[^a].|..[^t])$", 'autoexec.bat')

In [76]:
re.match(r".*[.]([^b]..|.[^a].|..[^t])$", 'sendmail.cf')

In [77]:
re.match(r".*[.]([^b]..|.[^a].|..[^t])$", 'printers.conf')

In [78]:
re.match(r".*[.][^(bat)].*$", 'foo.bar')

In [79]:
re.match(r".*[.][^(bat)].*$", 'sendmail.cf')

<re.Match object; span=(0, 11), match='sendmail.cf'>

In [86]:
re.match(r".*[^(bat)].*", 'adbt')

<re.Match object; span=(0, 4), match='adbt'>

In [89]:
re.match(r".*[^(bat)].*", 'abatba')

一个好的解决办法
negative lookahead

In [3]:
files = """
foo.bar
autoexec.bat
sendmail.cf
printers.conf
"""

In [4]:
import re

p = re.compile(r".*[.](?!bat$)[^.]*$", re.MULTILINE)
p.findall(files)

['foo.bar', 'sendmail.cf', 'printers.conf\n']

以下正则表达式模式排除所有以 `bat` 和 `exe` 结尾的文件名。


`.*[.](?!bat$|exe$)[^.]*$`

In [5]:
files = """
foo.bar
autoexec.bat
sendmail.cf
printers.conf
foo.bat.exe
foo.bat.cf
foo.cf.bat
foo.exe.cf
"""

In [6]:
p = re.compile(r".*[.](?!bat$|exe$)[^.]*$", re.MULTILINE)
p.findall(files)

['foo.bar', 'sendmail.cf', 'printers.conf', 'foo.bat.cf', 'foo.exe.cf\n']

## modifying strings

正则表达式操作字符串的方式：

- `split()`，将字符串在符合正则表达式的地方切分成列表。
- `sub()`，查找替换（所有）。
- `subn()`，替换，并返回新的字符串和替换次数。 

### splitting strings

通过 `maxsplit=0` 限制分割次数。下面的例子中，分隔符是所有的非字母字符。

In [7]:
p = re.compile(r'\W+')
p.split('This is a test, short and sweet, of split().')

['This', 'is', 'a', 'test', 'short', 'and', 'sweet', 'of', 'split', '']

In [8]:
p.split('This is a test, short and sweet, of split().', 3)

['This', 'is', 'a', 'test, short and sweet, of split().']

In [9]:
p.split('This is a test, short and sweet, of split().', maxsplit=3)

['This', 'is', 'a', 'test, short and sweet, of split().']

当正则表达模式用圆括号括起来时，匹配到的内容也会作为返回值出现在 `split()` 方法的结果中。

In [10]:
p = re.compile(r'\W+')
p2 = re.compile(r'(\W+)')

In [12]:
p.split('This... is a test.')

['This', 'is', 'a', 'test', '']

In [14]:
p.split('This ... is a test.')

['This', 'is', 'a', 'test', '']

In [13]:
p2.split('This... is a test.')

['This', '... ', 'is', ' ', 'a', ' ', 'test', '.', '']

模块级方法 split

In [15]:
re.split(r'[\W+]', 'Words, words, words.')

['Words', '', 'words', '', 'words', '']

In [16]:
re.split(r'\W+', 'Words, words, words.')

['Words', 'words', 'words', '']

> 上面这两个搞不太懂。

In [17]:
re.split(r'(\W+)', 'Words, words, words.')

['Words', ', ', 'words', ', ', 'words', '.', '']

In [18]:
re.split(r'([\W])+', 'Words, words, words')

['Words', ' ', 'words', ' ', 'words']

In [19]:
re.split(r'[\W]+', 'Words, words, words.')

['Words', 'words', 'words', '']

> 上面这几个都看不懂。

### search and replace

In [20]:
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes')

'colour socks and colour shoes'

In [21]:
p.sub('colour', 'blue socks and red shoes', count=1)

'colour socks and red shoes'

In [22]:
p.subn('colour', 'blue socks and red shoes')

('colour socks and colour shoes', 2)

In [23]:
p.subn('colour', 'blue socks and red shoes', count=1)

('colour socks and red shoes', 1)

Empty matches are replaced only when they’re not adjacent to a previous empty match.

这句话没看懂。

大概是说 b 和 d 中间有 x，然后替换了 两个 --，具体还是不懂。

In [25]:
p = re.compile('x*')  # * 表示0个或多个
p.sub('-', 'abxd')

'-a-b--d-'

In [26]:
p = re.compile('section{ ( [^}]* ) }', re.VERBOSE)
p.sub(r'subsection{\1}', 'section{First} section{second} subsection{third}')

'subsection{First} subsection{second} subsubsection{third}'

In [27]:
p = re.compile('section{ (?P<name> [^}]* ) }', re.VERBOSE)
p.sub(r'subsection{\1}', 'section{First}')

'subsection{First}'

In [28]:
p.sub(r'subsectio{\g<1>}', 'section{First}')

'subsectio{First}'

In [30]:
p.sub(r'subsectio{\g<0>}', 'section{First}')

'subsectio{section{First}}'

In [33]:
p = re.compile('section{ (?P<name> [^}]* ) }', re.VERBOSE)
p.sub(r'subsection{\g<name>}', 'section{First}')
# 这里可以用 name，是因为在 compile 里面也给了组的名字是 name

'subsection{First}'

当 sub() 函数的 replacement 参数是一个函数的时候。

下面的例子中，replacement 处的函数将十进制数转换为十六进制数。

In [35]:
def hexrepl(match):
    """Return the hex string for a decimal number."""
    value = int(match.group())
    return hex(value)

In [36]:
p = re.compile(r'\d+')
p.sub(hexrepl, 'Call 65490 for printing, 49512 for user code.')

'Call 0xffd2 for printing, 0xc168 for user code.'

In [34]:
hex(15)

'0xf'

In [52]:
p = re.compile(r'\d+')
p.search('Call 65490 for printing, 49512 for user code.')
# serach 找到一次就结束

<re.Match object; span=(5, 10), match='65490'>

还有 inline flag 这种东西存在

In [53]:
re.sub("(?i)b+", 'x', 'bbbb BBBB')
# `(?i)` 就表示 re.IGNORECASE

'x x'

In [54]:
re.sub("b+", 'x', 'bbbb BBBB')

'x BBBB'

## common problems

有时候我们无法让正则表达式按照我们期望的那样去实现效果。

### use string methods

如果只是匹配某个确定字符串，或某个字符类别，就不需要用 re 模块。可以使用 [`replace()`](https://docs.python.org/3/library/stdtypes.html#str.replace) method。

`replace()` 也会把单词内的字符给替换掉。

另一个是做简单的字符替换，可以考虑 `str.translate()` 【但是这个方法没看懂】

In [55]:
files

'\nfoo.bar\nautoexec.bat\nsendmail.cf\nprinters.conf\nfoo.bat.exe\nfoo.bat.cf\nfoo.cf.bat\nfoo.exe.cf\n'

### match() versus search()

`match()` 只检查从字符串开始的匹配情况；

`search()` 从头开始检查字符串，返回所有匹配结果。

In [60]:
re.match('super', 'superstition').span()

(0, 5)

In [61]:
re.match('super', 'insuperable')

In [62]:
re.search('super', 'superstition').span()

(0, 5)

In [63]:
re.search('super', 'insuperable').span()

(2, 7)

### greedy versus non-greedy

不懂说的啥，大概理解贪婪的意思就是尽可能多地匹配。

In [64]:
s = '<html><head><title>Title</title>'
len(s)

32

In [65]:
re.match('<.*>', s).span()

(0, 32)

In [66]:
re.match('<.*>', s).group()

'<html><head><title>Title</title>'

In [67]:
re.match('<.*>', s).groups()

()

好像懂了，本来是准备匹配 html 中的标签的。那要怎么做呢。

non-greedy qualifiers

```
*? +? ?? {m,n}
```

这些符号会尽可能匹配更少文本。

In [69]:
re.match('<.*?>', s).group()

'<html>'

解析 html 或 xml 格式文件最好的办法还是用第三方包，自己写解析方法会让你头秃。

### using re.VERBOSE

简短的正则表达式没关系，但是太过复杂的就让人看着难受，这时使用 verbose flag并写注释就很有用了。



In [73]:
pat = re.compile(r"""
 \s*                # skip leading whitespace
 (?P<header>[^:]+)  # header name
 \s* :              # whitespace, and a colon
 (?P<value>.*?)     # the header's value  -- *? used to 
                    # lose the following trailing whitespace
 \s*$               # Trailing whitespace to end-of-line
""", re.VERBOSE)

对应的非 verbose 的形式是

In [79]:
pat = re.compile(r"\s*(?P<header>[^:]+)\s*:(?P<value>|.*?)\s*$")

这个还要看看是匹配啥的。

In [77]:
pat = re.compile(r"\s*(?P<header>[^:]+)\s*:(?P<value>.*?)\s*$")

In [80]:
pat.search('afafaef:egegeg')

<re.Match object; span=(0, 14), match='afafaef:egegeg'>

In [81]:
pat.search('afafaef  : 123  ')

<re.Match object; span=(0, 16), match='afafaef  : 123  '>